# 
Initial Data Exploration – Online Retail Dataset

## Objective
This notebook performs an initial exploratory analysis of the raw Online Retail dataset.
The goal is to understand:
- Dataset structure and size
- Data quality issues (missing values, duplicates, anomalies)
- Transactional patterns
- Potential challenges for downstream cleaning and modeling

⚠️ This notebook performs **read-only analysis**.
No data cleaning is done here.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

print("Libraries loaded successfully")


Libraries loaded successfully


In [2]:
df = pd.read_excel(
    "../data/raw/online_retail_II.xlsx",
    engine="openpyxl",
    parse_dates=["InvoiceDate"]
)

print(f"Dataset Shape: {df.shape}")
df.head(10)


Dataset Shape: (525461, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom


## Dataset Overview

Each row represents a single transaction line item.
The dataset captures customer purchase behavior across multiple countries
over a multi-month period.

Next, we analyze basic statistics and data quality issues.


In [4]:
start_date = df["InvoiceDate"].min()
end_date = df["InvoiceDate"].max()

unique_customers = df["Customer ID"].nunique()
unique_products = df["StockCode"].nunique()
unique_countries = df["Country"].nunique()

print("Basic Statistics")
print("-" * 40)
print(f"Date Range: {start_date} → {end_date}")
print(f"Unique Customers: {unique_customers}")
print(f"Unique Products: {unique_products}")
print(f"Unique Countries: {unique_countries}")


Basic Statistics
----------------------------------------
Date Range: 2009-12-01 07:45:00 → 2010-12-09 20:01:00
Unique Customers: 4383
Unique Products: 4632
Unique Countries: 40


## Data Quality Issues

We now examine missing values, duplicate records,
and anomalies that may affect analysis and modeling.


In [8]:
missing_values = df.isnull().sum()
missing_customer_pct = (df["Customer ID"].isnull().sum() / len(df)) * 100

print("Missing Values Per Column")
print(missing_values)

print(f"\nMissing Customer ID Percentage: {missing_customer_pct:.2f}%")


Missing Values Per Column
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Missing Customer ID Percentage: 20.54%


In [9]:
duplicate_rows = df.duplicated().sum()
print(f"Total Duplicate Rows: {duplicate_rows}")


Total Duplicate Rows: 6865


## Transaction Patterns

We analyze quantity and pricing behavior to detect
returns, cancellations, and data errors.


In [10]:
df["Quantity"].describe()


count    525461.000000
mean         10.337667
std         107.424110
min       -9600.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       19152.000000
Name: Quantity, dtype: float64

In [11]:
negative_quantities = (df["Quantity"] < 0).sum()
print(f"Negative Quantities (Returns): {negative_quantities}")


Negative Quantities (Returns): 12326


In [14]:
df["Price"].describe()


count    525461.000000
mean          4.688834
std         146.126914
min      -53594.360000
25%           1.250000
50%           2.100000
75%           4.210000
max       25111.090000
Name: Price, dtype: float64

In [15]:
zero_negative_prices = (df["Price"] <= 0).sum()
print(f"Zero or Negative Prices: {zero_negative_prices}")


Zero or Negative Prices: 3690


## Anomaly Detection

Invoice patterns and extreme values are analyzed
to identify cancellations and abnormal behavior.


In [18]:
cancelled_invoices = df["Invoice"].astype(str).str.startswith("C").sum()
print(f"Cancelled Invoices: {cancelled_invoices}")


Cancelled Invoices: 10206


In [19]:
high_qty_threshold = df["Quantity"].quantile(0.99)
extreme_quantities = (df["Quantity"] > high_qty_threshold).sum()

print(f"99th percentile quantity: {high_qty_threshold}")
print(f"Rows above 99th percentile: {extreme_quantities}")


99th percentile quantity: 120.0
Rows above 99th percentile: 4612


## Data Quality Summary

The following summary is saved as a JSON artifact.
This file is required for automated evaluation.


In [21]:
import json
from pathlib import Path

summary = {
    "total_rows": int(len(df)),
    "total_columns": int(len(df.columns)),
    "missing_values": missing_values.to_dict(),
    "duplicate_rows": int(duplicate_rows),
    "date_range": {
        "start": str(start_date),
        "end": str(end_date)
    },
    "negative_quantities": int(negative_quantities),
    "cancelled_invoices": int(cancelled_invoices),
    "missing_customer_ids": int(df["Customer ID"].isnull().sum()),
    "missing_customer_ids_percentage": round(missing_customer_pct, 2)
}

Path("../data/raw").mkdir(parents=True, exist_ok=True)

with open("../data/raw/data_quality_summary.json", "w") as f:
    json.dump(summary, f, indent=4)

print("data_quality_summary.json saved successfully")


data_quality_summary.json saved successfully


## Key Observations

- Dataset contains over 500k transaction rows.
- CustomerID has a significant number of missing values.
- Negative quantities and cancelled invoices indicate returns.
- Zero or negative prices suggest data quality issues.
- These findings justify a robust data cleaning phase.

➡️ Next: Data Validation Notebook
